In [17]:
using DataFrames
using CSV
using DecisionTree
using Statistics

In [45]:
d = DataFrame(CSV.File("/Users/psaroudakis/Downloads/g2f-maize-challenge-2022/processed_data/combined_mat.csv"))
x = d[:, 12:ncol(d)]
x = x[:, sum.(ismissing, eachcol(x)) .== 0]

# Remove string variables (cannot work with them so far)
x = x[!, Not(names(x, AbstractString))]

# Transform Strings to Categorical Variables
# transform!(x, names(x, AbstractString) .=> categorical, renamecols=false)

# Adjust yield to 15.5% moisture
y = d.Yield_Mg_ha - d.Yield_Mg_ha .* (d.Grain_Moisture .- 15.5)/100

x

Row,Plot_Area_ha,Yield_Mg_ha,ec_dta_AccumulatedTT_pEmeEnJ,ec_dta_AccumulatedTT_pEnGMat,ec_dta_AccumulatedTT_pEnJFlo,ec_dta_AccumulatedTT_pFlaFlw,ec_dta_AccumulatedTT_pFloFla,ec_dta_AccumulatedTT_pFlwStG,ec_dta_AccumulatedTT_pGerEme,ec_dta_AccumulatedTT_pMatHar,ec_dta_AccumulatedTT_pStGEnG,ec_dta_CoverGreen_pEmeEnJ,ec_dta_CoverGreen_pEnGMat,ec_dta_CoverGreen_pEnJFlo,ec_dta_CoverGreen_pFlaFlw,ec_dta_CoverGreen_pFloFla,ec_dta_CoverGreen_pFlwStG,ec_dta_CoverGreen_pGerEme,ec_dta_CoverGreen_pMatHar,ec_dta_CoverGreen_pStGEnG,ec_dta_CoverTotal_pEmeEnJ,ec_dta_CoverTotal_pEnGMat,ec_dta_CoverTotal_pEnJFlo,ec_dta_CoverTotal_pFlaFlw,ec_dta_CoverTotal_pFloFla,ec_dta_CoverTotal_pFlwStG,ec_dta_CoverTotal_pGerEme,ec_dta_CoverTotal_pMatHar,ec_dta_CoverTotal_pStGEnG,ec_dta_ESW_pEmeEnJ_1,ec_dta_ESW_pEmeEnJ_10,ec_dta_ESW_pEmeEnJ_2,ec_dta_ESW_pEmeEnJ_3,ec_dta_ESW_pEmeEnJ_4,ec_dta_ESW_pEmeEnJ_5,ec_dta_ESW_pEmeEnJ_6,ec_dta_ESW_pEmeEnJ_7,ec_dta_ESW_pEmeEnJ_8,ec_dta_ESW_pEmeEnJ_9,ec_dta_ESW_pEnGMat_1,ec_dta_ESW_pEnGMat_10,ec_dta_ESW_pEnGMat_2,ec_dta_ESW_pEnGMat_3,ec_dta_ESW_pEnGMat_4,ec_dta_ESW_pEnGMat_5,ec_dta_ESW_pEnGMat_6,ec_dta_ESW_pEnGMat_7,ec_dta_ESW_pEnGMat_8,ec_dta_ESW_pEnGMat_9,ec_dta_ESW_pEnJFlo_1,ec_dta_ESW_pEnJFlo_10,ec_dta_ESW_pEnJFlo_2,ec_dta_ESW_pEnJFlo_3,ec_dta_ESW_pEnJFlo_4,ec_dta_ESW_pEnJFlo_5,ec_dta_ESW_pEnJFlo_6,ec_dta_ESW_pEnJFlo_7,ec_dta_ESW_pEnJFlo_8,ec_dta_ESW_pEnJFlo_9,ec_dta_ESW_pFlaFlw_1,ec_dta_ESW_pFlaFlw_10,ec_dta_ESW_pFlaFlw_2,ec_dta_ESW_pFlaFlw_3,ec_dta_ESW_pFlaFlw_4,ec_dta_ESW_pFlaFlw_5,ec_dta_ESW_pFlaFlw_6,ec_dta_ESW_pFlaFlw_7,ec_dta_ESW_pFlaFlw_8,ec_dta_ESW_pFlaFlw_9,ec_dta_ESW_pFloFla_1,ec_dta_ESW_pFloFla_10,ec_dta_ESW_pFloFla_2,ec_dta_ESW_pFloFla_3,ec_dta_ESW_pFloFla_4,ec_dta_ESW_pFloFla_5,ec_dta_ESW_pFloFla_6,ec_dta_ESW_pFloFla_7,ec_dta_ESW_pFloFla_8,ec_dta_ESW_pFloFla_9,ec_dta_ESW_pFlwStG_1,ec_dta_ESW_pFlwStG_10,ec_dta_ESW_pFlwStG_2,ec_dta_ESW_pFlwStG_3,ec_dta_ESW_pFlwStG_4,ec_dta_ESW_pFlwStG_5,ec_dta_ESW_pFlwStG_6,ec_dta_ESW_pFlwStG_7,ec_dta_ESW_pFlwStG_8,ec_dta_ESW_pFlwStG_9,ec_dta_ESW_pGerEme_1,ec_dta_ESW_pGerEme_10,ec_dta_ESW_pGerEme_2,ec_dta_ESW_pGerEme_3,ec_dta_ESW_pGerEme_4,ec_dta_ESW_pGerEme_5,ec_dta_ESW_pGerEme_6,ec_dta_ESW_pGerEme_7,ec_dta_ESW_pGerEme_8,ec_dta_ESW_pGerEme_9,ec_dta_ESW_pMatHar_1,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.000812902,6.96771,183.854,1498.44,281.12,849.247,559.54,932.67,56.4986,1507.15,1246.84,0.00702324,0.493278,0.0262391,0.728194,0.434156,0.710689,6.61516e-5,0.48695,0.610433,0.00702324,0.51889,0.0262391,0.72959,0.434362,0.713528,6.61516e-5,0.513333,0.623028,33.2124,12.735,38.1677,37.3446,21.1008,14.7018,13.6822,9.3926,15.0529,12.9063,2.99345,58.2076,12.0782,15.9432,15.6891,18.0362,16.8914,15.7875,21.2206,24.9504,52.0547,12.7585,48.7339,41.5423,31.9471,14.9821,13.7529,9.29358,15.1842,12.9609,43.3417,56.253,44.9899,45.0839,43.214,41.8756,29.8491,28.9953,30.4231,30.9999,44.6534,19.8297,47.2744,45.9909,43.1361,38.1485,26.3786,22.2921,23.6577,20.1678,34.6873,58.6921,37.6094,38.7989,38.7234,39.3689,28.9404,28.9099,30.2922,30.6149,31.3261,12.7029,36.9969,35.1077,15.6633,14.7534,13.5785,9.54369,14.8574,12.8316,2.39506,⋯
2,0.000812902,8.54328,183.854,1498.44,281.12,849.247,559.54,932.67,56.4986,1507.15,1246.84,0.00702324,0.493278,0.

In [42]:
import JSON

split = JSON.parsefile("/Users/psaroudakis/Downloads/g2f-maize-challenge-2022/processed_data/train_test_split_v2.json")

Dict{String, Any} with 70 entries:
  "2018@2" => Dict{String, Any}("test"=>Any[19721, 19723, 19729, 19736, 19745, …
  "2015@3" => Dict{String, Any}("test"=>Any[6, 12, 14, 16, 18, 19, 39, 44, 53, …
  "2016@5" => Dict{String, Any}("test"=>Any[4789, 4790, 4800, 4804, 4805, 4807,…
  "2015@4" => Dict{String, Any}("test"=>Any[2, 10, 13, 18, 20, 28, 31, 35, 37, …
  "2017@6" => Dict{String, Any}("test"=>Any[13857, 13859, 13863, 13871, 13872, …
  "2019@2" => Dict{String, Any}("test"=>Any[28915, 28919, 28921, 28932, 28938, …
  "2020@4" => Dict{String, Any}("test"=>Any[42987, 42988, 42989, 42990, 42997, …
  "2017@1" => Dict{String, Any}("test"=>Any[13866, 13877, 13889, 13897, 13898, …
  "2019@5" => Dict{String, Any}("test"=>Any[28924, 28931, 28936, 28939, 28945, …
  "2016@0" => Dict{String, Any}("test"=>Any[4789, 4793, 4813, 4816, 4817, 4821,…
  "2016@1" => Dict{String, Any}("test"=>Any[4790, 4795, 4803, 4820, 4824, 4825,…
  "2019@3" => Dict{String, Any}("test"=>Any[28916, 28923, 28928, 28932, 28

In [69]:
rmses = DataFrame(Set_name = String[], RMSE = Float64[])

loop_i = 0

# @TODO potentially parallelize this with `addprocs()`?

for(key, val) in split
    # @TODO this is just in here to stop after 3 sets
    if loop_i > 2
        break
    end
    
    println(key)
    x_train = x[convert(Vector{Int}, val["train"] .+ 1), :]
    y_train = y[convert(Vector{Int}, val["train"] .+ 1)]
    x_test  = x[convert(Vector{Int}, val["test"] .+ 1), :]
    y_test  = y[convert(Vector{Int}, val["test"] .+ 1)]
    
    # We're using R defaults here
    model = build_forest(y_train, Matrix(x_train), ceil(ncol(x)/3), 500, 0.632)
    print(model)
    preds = apply_forest(model, Matrix(x_test))

    push!(rmses, [key, sqrt(mean((y_test - preds) .^ 2))])
    loop_i += 1
end
CSV.write("/Users/psaroudakis/Downloads/g2f-maize-challenge-2022/modeling/dennis/1_rmses.csv", rmses)
rmses

2018@2


LoadError: TaskFailedException

[91m    nested task error: [39mInterruptException:
    Stacktrace:
      [1] [0m[1mArray[22m
    [90m    @ [39m[90m./[39m[90m[4mboot.jl:459[24m[39m[90m [inlined][39m
      [2] [0m[1mArray[22m
    [90m    @ [39m[90m./[39m[90m[4mboot.jl:468[24m[39m[90m [inlined][39m
      [3] [0m[1msimilar[22m
    [90m    @ [39m[90m./[39m[90m[4marray.jl:378[24m[39m[90m [inlined][39m
      [4] [0m[1msimilar[22m
    [90m    @ [39m[90m./[39m[90m[4mabstractarray.jl:795[24m[39m[90m [inlined][39m
      [5] [0m[1mgetindex[22m
    [90m    @ [39m[90m./[39m[90m[4marray.jl:932[24m[39m[90m [inlined][39m
      [6] [0m[1m_split![22m[0m[1m([22m[90mX[39m::[0mMatrix[90m{Float64}[39m, [90mY[39m::[0mVector[90m{Float64}[39m, [90mW[39m::[0mVector[90m{Float64}[39m, [90mnode[39m::[0mDecisionTree.treeregressor.NodeMeta[90m{Float64}[39m, [90mmax_features[39m::[0mInt64, [90mmax_depth[39m::[0mInt64, [